In [ ]:
import osmnx as ox
import pandas as pd
import pvlib 
from datetime import datetime, date, time, timedelta 
import utm
import math


superf_techo = {'surface_tilt': 0} 
superf_derecha = {'surface_tilt': 90} 
superf_izquierda = {'surface_tilt': 90} 
superf_capo = {'surface_tilt': 15} 
superf_maletero = {'surface_tilt': 45} 
coord_origen = (40.462834, -3.585214)
coord_destino = (40.458570, -3.861030) 
timezone = 'Etc/GMT+2'
altitude=667
hora_salida = datetime(2020, 7, 28, 14, 30, 0) #y, m, d, h, min, s 

In [ ]:
superficies_coche =[]
superficies_coche.append(superf_techo)
superficies_coche.append(superf_derecha)
superficies_coche.append(superf_izquierda)
superficies_coche.append(superf_capo)
superficies_coche.append(superf_maletero)

In [ ]:
orig_dest=[]
dist_to_utm=[]
dist_list=[]
dist_tuple=[]

orig_dest.append(coord_origen)
orig_dest.append(coord_destino)

dist_to_utm.append(utm.from_latlon(orig_dest[0][0],orig_dest[0][1]))
dist_to_utm.append(utm.from_latlon(orig_dest[1][0],orig_dest[1][1]))
     
dist_list.append(dist_to_utm[0][0])
dist_list.append(dist_to_utm[0][1])
dist_list.append(dist_to_utm[1][0])
dist_list.append(dist_to_utm[1][1])

for x in range(0, len(dist_list), 2):
    dist_tuple.append(dist_list[x:x+2])

distancia = math.sqrt((dist_tuple[1][0]-dist_tuple[0][0])**2+(dist_tuple[1][1]-dist_tuple[0][1])**2)

In [ ]:
mapa_nodos = ox.graph_from_point (coord_origen, dist=distancia, dist_type ='bbox' , network_type = 'drive')
fig, ax = ox.plot_graph(mapa_nodos) 
ox.speed.add_edge_speeds(mapa_nodos , hwy_speeds = 30 , fallback = 30 , precision = 0 )
ox.speed.add_edge_travel_times( mapa_nodos , precision = 0 )

In [ ]:
# #GRAFICAR A GEODATAFRAME
nodos, caminos = ox.graph_to_gdfs(mapa_nodos) #Función de OSMnx que convierte el multidigraph en geodataframes de NODOS y LÍNEAS
# pd.set_option("max_columns", None) #Showing only two columns
# pd.set_option("max_rows", None)

In [ ]:
#------------------SECCIÓN INFORMATIVA-----------------------------#
nodos.head() #GeoDataFrames de los nodos
caminos.head()

In [ ]:
#---------------------NODOS ORIGEN Y DESTINO SEGÚN COORDENADAS----------------------------------#
nodo_origen = ox.get_nearest_node(mapa_nodos, coord_origen, method='haversine', return_dist=False)
nodo_destino = ox.get_nearest_node(mapa_nodos, coord_destino, method='haversine', return_dist=False)

In [ ]:
#--------------------------NODOS DE LA RUTA---------------------------------------#
ruta = ox.shortest_path(G=mapa_nodos, orig=nodo_origen, dest=nodo_destino, weight='length')

In [ ]:
#--------------------------COORDENADAS TOTALES EN UNA LISTA---------------------------------------#
lista_coords_nodos =[]
for it_coords in ruta:
    lista_coords_nodos.append(mapa_nodos.nodes[it_coords]['y'])
    lista_coords_nodos.append(mapa_nodos.nodes[it_coords]['x'])
    
#--------------------------COORDENADAS EMPAREJADAS---------------------------------------#
coords_nodos = []
for i in range(0, len(lista_coords_nodos), 2):
    coords_nodos.append(lista_coords_nodos[i:i+2])

#--------------------------COORDENADAS TUPLE---------------------------------------#
coords_list=[]
for it_xy in coords_nodos:
    coords_list.append(tuple(it_xy))

In [ ]:
#----------------------------TRAZADO DE LA RUTA-------------------------------------------#
caminos.plot(ax=ax, linewidth=1, edgecolor='dimgray') #Líneas
fig2, ax = ox.plot_graph_route(mapa_nodos, ruta)

In [ ]:
#-------------------------INFORMACIÓN POR NODO DE LA RUTA----------------------------------------#
ruta_info=ox.utils_graph.get_route_edge_attributes(mapa_nodos, ruta, attribute=None, minimize_key='length', retrieve_default=None)

In [ ]:
distancias=[]
for it_dist in ruta_info:
    distancias.append(it_dist['length'])
distancias[32]
len(distancias)

In [ ]:
#-------VELOCIDAD-----# 
speed_info=[] #max_speed
for it_speed in ruta_info:
    if 'maxspeed' in it_speed:
        speed_info.append(it_speed['maxspeed'])
    else:
        speed_info.append('65')
        
speed_to_float=[] #max_speed_fl
for it_speed_fl in speed_info: 
    if type(it_speed_fl) == str:
        if len(it_speed_fl) > 6:
            speed_to_float.append(float(it_speed_fl[0:3]))
        elif len(it_speed_fl) > 3 and len(it_speed_fl) < 6:
                speed_to_float.append(float(it_speed_fl[0:2]))
        else:
            speed_to_float.append(float(it_speed_fl))
    elif type(it_speed_fl) == list:
        speed_to_float.append(float(it_speed_fl[0]))  
        
len(speed_to_float)

In [ ]:
# speed_to_float=[] #max_speed_fl
# for it_speed_fl in speed_info: 
#     if type(it_speed_fl) == str:
#         speed_to_float.append(float(it_speed_fl))
#     elif type(it_speed_fl) == list:
#         speed_to_float.append(float(it_speed_fl[0]))
# speed_to_float
# speed_to_float[32]

In [ ]:
#-------TIEMPO-----#
time=[]
for x in range(0,len(ruta_info)):
    time.append(round(distancias[x]/(speed_to_float[x]*1000/3600)))

In [ ]:
#---------HORA DE PASADA POR NODO-----------#
horas_intermedias = hora_salida
hora_llegada=hora_salida #Únicamente para un dato final
horas_trayecto = []
for it_trayecto in time:
    horas_trayecto.append(horas_intermedias + timedelta(seconds=it_trayecto))
    horas_intermedias += timedelta(seconds=it_trayecto)
    
for it_llegada in time:
    hora_llegada+=timedelta(seconds=it_llegada)

#-------------FECHAS ADAPTADAS PARA EL ANÁLISIS-----------#
datetimes=[]
for x in range(1,len(horas_trayecto)):
    datetimes.append(pd.date_range(start=horas_trayecto[x-1], end=horas_trayecto[x]- timedelta(seconds=1), freq='1S'))


#------DATAFRAME--------#
horas_trayecto_pd =[]
for x in range(1,len(horas_trayecto)):
    for i in range(0,len(datetimes[x-1])): 
         horas_trayecto_pd.append(datetimes[x-1][i])
df = pd.DataFrame(horas_trayecto_pd,columns=['Hora'])
df.set_index('Hora', inplace = True)

In [ ]:
#--------------------------NOMBRE DE LAS CALLES---------------------------------------#
nombre_calles =[]
for it_direccion in ruta_info:
     if 'name' in it_direccion:
        nombre_calles.append(it_direccion['name'])
     else:
        nombre_calles.append(" ")
    


#------DATAFRAME--------#
nombre_calles_pd=[]
for x in range(1,len(horas_trayecto)):
    for i in range(0,len(datetimes[x-1])):
        nombre_calles_pd.append(nombre_calles[x-1])
df['Dirección'] = nombre_calles_pd

In [ ]:
#------DATAFRAME--------#
time_pd=[]
for x in range(1,len(horas_trayecto)):
    for i in range(0,len(datetimes[x-1])):
        time_pd.append(time[x])
df['Tiempo'] = time_pd

In [ ]:
#--------------------------NODOS INICIALES DE LAS CALLES---------------------------------------#
nodo1_calle =[]
for x in range(0,len(ruta)-1):
    nodo1_calle.append(ruta[x])
    
#------DATAFRAME--------#
nodo1_calle_pd=[]
for x in range(1,len(horas_trayecto)):
    for i in range(0,len(datetimes[x-1])):
        nodo1_calle_pd.append(nodo1_calle[x-1])
df['Nodo Origen'] = nodo1_calle_pd

In [ ]:
#--------------------------COORDENADAS INICIALES DE LAS CALLES---------------------------------------#
coords1_calle =[]
for x in range(0,len(ruta)-1): #Elimino la ultima coordenada para tener longitud 30
    coords1_calle.append(coords_nodos[x])
    
#------DATAFRAME--------#
coords1_calle_pd=[]
for x in range(1,len(horas_trayecto)):
    for i in range(0,len(datetimes[x-1])):
       coords1_calle_pd.append(coords_list[x-1]) 
df['Origen'] = coords1_calle_pd

In [ ]:
#--------------------------NODOS FINALES DE LAS CALLES---------------------------------------#
nodo2_calle =[]
for x in range(1,len(ruta)):
    nodo2_calle.append(ruta[x])
    
#------DATAFRAME--------#
nodo2_calle_pd=[]
for x in range(1,len(horas_trayecto)):
    for i in range(0,len(datetimes[x-1])):
        nodo2_calle_pd.append(nodo2_calle[x-1])
df['Nodo Destino'] = nodo2_calle_pd

In [ ]:
#-------------------------COORDENADAS FINALES DE LAS CALLES--------------------------------------#
coords2_calle =[]
for x in range(1,len(ruta)):
    coords2_calle.append(coords_nodos[x])
    
#------DATAFRAME--------#
coords2_calle_pd=[]
for x in range(1,len(horas_trayecto)):
    for i in range(0,len(datetimes[x-1])):
      coords2_calle_pd.append(coords_list[x])    
df['Destino'] = coords2_calle_pd

In [ ]:
#----------------------------VELOCIDAD POR CALLE EN KPH-------------------------------------------#
calles_speed =[]
for it_speed in speed_to_float:
    calles_speed.append(it_speed)
    
#------DATAFRAME--------#
calles_speed_pd=[]
for x in range(1,len(horas_trayecto)):
    for i in range(0,len(datetimes[x-1])):
        calles_speed_pd.append(calles_speed[x-1])
df['km/h'] = calles_speed_pd

In [ ]:
#-------ACIMUT SEGÚN LOS GIROS DEL COCHE------#
import utm
import numpy as np
import math
p0 = [0, 0]
p1 = (0, 1)
v0 = np.array(p1) - np.array(p0) #FIJA

coords_to_utm=[]
utm_list=[]
utm_tuple=[]
utm_resta=[]
coords_acimut=[]



for x in range(1,len(coords_list)):
    coords_to_utm.append(utm.from_latlon(coords_list[x-1][0],coords_list[x-1][1]))
    
for x in range(1,len(coords_list)):
    utm_list.append(coords_to_utm[x-1][0])
    utm_list.append(coords_to_utm[x-1][1])

for x in range(0, len(utm_list), 2):
    utm_tuple.append(utm_list[x:x+2])

for x in range(1,len(coords_list)-1):
    utm_resta.append(np.array(utm_tuple[x]) - np.array(utm_tuple[x-1]))
    
for x in range(1,len(coords_list)-1):
    coords_acimut.append(round(math.degrees(np.math.atan2(np.linalg.det([utm_resta[x-1],v0]),np.dot(utm_resta[x-1],v0)))))


    
#------DATAFRAME--------#
angulo_pd=[]
for x in range(1,len(horas_trayecto)):
    for i in range(0,len(datetimes[x-1])):
        if coords_acimut[x-1] < 0:
            angulo_pd.append(coords_acimut[x-1]+360)
        else:
            angulo_pd.append(coords_acimut[x-1])
        
df['Azimuth']=angulo_pd

In [ ]:
solar_pos=[]
dni_extra=[]
airmass_rel =[]
pressure = pvlib.atmosphere.alt2pres(altitude)
airmass_abs = []
turbidez=[]
val_sol=[]
for x in range(1,len(horas_trayecto)):
    naive_times = pd.date_range(start=horas_trayecto[x-1], end=horas_trayecto[x]- timedelta(seconds=1), freq='1S')
    times = naive_times.tz_localize(timezone)
    solar_pos.append(pvlib.solarposition.get_solarposition(times, coords_list[x-1][0], coords_list[x-1][1]))
    dni_extra.append(pvlib.irradiance.get_extra_radiation(times))
    turbidez.append(pvlib.clearsky.lookup_linke_turbidity(times, coords_list[x-1][0], coords_list[x-1][1]))
    
for x in range(1,len(horas_trayecto)):
    airmass_rel.append(pvlib.atmosphere.get_relative_airmass(solar_pos[x-1]['apparent_zenith']))

for x in range(1,len(horas_trayecto)):
    airmass_abs.append(pvlib.atmosphere.get_absolute_airmass(airmass_rel[x-1], pressure))

for x in range(1,len(horas_trayecto)):
    val_sol.append(pvlib.clearsky.ineichen(solar_pos[x-1]['apparent_zenith'], airmass_abs[x-1], turbidez[x-1],
                                 dni_extra=dni_extra[x-1], altitude=altitude))


In [ ]:
#------------ TECHO----------------------#
irrad_techo=[]
for x in range (1,len(horas_trayecto)):
    irrad_techo.append(pvlib.irradiance.get_total_irradiance(superf_techo['surface_tilt'],coords_acimut[x-1], 
    solar_pos[x-1]['zenith'] , solar_pos[x-1]['azimuth'], val_sol[x-1]['dni'], val_sol[x-1]['ghi'] , 
    val_sol[x-1]['dhi'], dni_extra=dni_extra[x-1], airmass=airmass_abs[x-1], albedo=0.10,
    surface_type=None, model='isotropic', model_perez='allsitescomposite1990'))
len(irrad_techo)
poa_global_filtro_techo=[] #Filtramos la parte de poa_global
df_global_techo=[]
poa_global_lista_techo=[]
for x in range(1,len(horas_trayecto)):
    poa_global_filtro_techo.append(irrad_techo[x-1]['poa_global'])
    df_global_techo.append(pd.DataFrame(poa_global_filtro_techo[x-1]))
    
for x in range(1,len(horas_trayecto)):   
    poa_global_lista_techo.append(df_global_techo[x-1].iloc[:, 0]) # Primera columna
    
poa_global_techo=[] 
for x in range(1,len(horas_trayecto)): 
    poa_global_techo.append(poa_global_lista_techo[x-1])
    
#------DATAFRAME--------#
poa_global_techo_pd=[]
for x in range(1,len(horas_trayecto)):
    for i in range(0,len(datetimes[x-1])):
        poa_global_techo_pd.append(poa_global_techo[x-1][i])
df['Techo'] = poa_global_techo_pd

In [ ]:
#------------ CAPÓ----------------------#
irrad_capo=[]
for x in range (1,len(horas_trayecto)):
    irrad_capo.append(pvlib.irradiance.get_total_irradiance(superf_capo['surface_tilt'],coords_acimut[x-1], 
    solar_pos[x-1]['zenith'] , solar_pos[x-1]['azimuth'], val_sol[x-1]['dni'], val_sol[x-1]['ghi'] , 
    val_sol[x-1]['dhi'], dni_extra=dni_extra[x-1], airmass=airmass_abs[x-1], albedo=0.10,
    surface_type=None, model='isotropic', model_perez='allsitescomposite1990'))
len(irrad_capo)
poa_global_filtro_capo=[] #Filtramos la parte de poa_global
df_global_capo=[]
poa_global_lista_capo=[]
for x in range(1,len(horas_trayecto)):
    poa_global_filtro_capo.append(irrad_capo[x-1]['poa_global'])
    df_global_capo.append(pd.DataFrame(poa_global_filtro_capo[x-1]))
    
for x in range(1,len(horas_trayecto)):   
    poa_global_lista_capo.append(df_global_capo[x-1].iloc[:, 0]) # Primera columna
    
poa_global_capo=[] 
for x in range(1,len(horas_trayecto)): 
    poa_global_capo.append(poa_global_lista_capo[x-1])
    
#------DATAFRAME--------#
poa_global_capo_pd=[]
for x in range(1,len(horas_trayecto)):
    for i in range(0,len(datetimes[x-1])):
        poa_global_capo_pd.append(poa_global_capo[x-1][i])
df['Capó'] = poa_global_capo_pd

In [ ]:
#------------ MALETERO----------------------#
irrad_maletero=[]
for x in range (1,len(horas_trayecto)):
    irrad_maletero.append(pvlib.irradiance.get_total_irradiance(superf_maletero['surface_tilt'],coords_acimut[x-1]+180, 
    solar_pos[x-1]['zenith'] , solar_pos[x-1]['azimuth'], val_sol[x-1]['dni'], val_sol[x-1]['ghi'] , 
    val_sol[x-1]['dhi'], dni_extra=dni_extra[x-1], airmass=airmass_abs[x-1], albedo=0.10,
    surface_type=None, model='isotropic', model_perez='allsitescomposite1990'))
len(irrad_maletero)
poa_global_filtro_maletero=[] #Filtramos la parte de poa_global
df_global_maletero=[]
poa_global_lista_maletero=[]
for x in range(1,len(horas_trayecto)):
    poa_global_filtro_maletero.append(irrad_maletero[x-1]['poa_global'])
    df_global_maletero.append(pd.DataFrame(poa_global_filtro_maletero[x-1]))
    
for x in range(1,len(horas_trayecto)):   
    poa_global_lista_maletero.append(df_global_maletero[x-1].iloc[:, 0]) # Primera columna
    
poa_global_maletero=[] 
for x in range(1,len(horas_trayecto)): 
    poa_global_maletero.append(poa_global_lista_maletero[x-1])
    
#------DATAFRAME--------#
poa_global_maletero_pd=[]
for x in range(1,len(horas_trayecto)):
    for i in range(0,len(datetimes[x-1])):
         poa_global_maletero_pd.append(poa_global_maletero[x-1][i])
df['Maletero'] = poa_global_maletero_pd

In [ ]:
#------------ PARED DERECHA----------------------#
irrad_derecha=[]
for x in range (1,len(horas_trayecto)):
    irrad_derecha.append(pvlib.irradiance.get_total_irradiance(superf_derecha['surface_tilt'],coords_acimut[x-1]+90, 
    solar_pos[x-1]['zenith'] , solar_pos[x-1]['azimuth'], val_sol[x-1]['dni'], val_sol[x-1]['ghi'] , 
    val_sol[x-1]['dhi'], dni_extra=dni_extra[x-1], airmass=airmass_abs[x-1], albedo=0.10,
    surface_type=None, model='isotropic', model_perez='allsitescomposite1990'))
len(irrad_derecha)
poa_global_filtro_derecha=[] #Filtramos la parte de poa_global
df_global_derecha=[]
poa_global_lista_derecha=[]
for x in range(1,len(horas_trayecto)):
    poa_global_filtro_derecha.append(irrad_derecha[x-1]['poa_global'])
    df_global_derecha.append(pd.DataFrame(poa_global_filtro_derecha[x-1]))
    
for x in range(1,len(horas_trayecto)):   
    poa_global_lista_derecha.append(df_global_derecha[x-1].iloc[:, 0]) # Primera columna
    
poa_global_derecha=[] 
for x in range(1,len(horas_trayecto)): 
    poa_global_derecha.append(poa_global_lista_derecha[x-1])
    
#------DATAFRAME--------#
poa_global_derecha_pd=[]
for x in range(1,len(horas_trayecto)):
    for i in range(0,len(datetimes[x-1])):
        poa_global_derecha_pd.append(poa_global_derecha[x-1][i])
df['Derecha'] = poa_global_derecha_pd

In [ ]:
#------------ PARED IZQUIERDA----------------------#
irrad_izquierda=[]
for x in range (1,len(horas_trayecto)):
    irrad_izquierda.append(pvlib.irradiance.get_total_irradiance(superf_izquierda['surface_tilt'],coords_acimut[x-1]-90, 
    solar_pos[x-1]['zenith'] , solar_pos[x-1]['azimuth'], val_sol[x-1]['dni'], val_sol[x-1]['ghi'] , 
    val_sol[x-1]['dhi'], dni_extra=dni_extra[x-1], airmass=airmass_abs[x-1], albedo=0.10,
    surface_type=None, model='isotropic', model_perez='allsitescomposite1990'))
len(irrad_izquierda)
poa_global_filtro_izquierda=[] #Filtramos la parte de poa_global
df_global_izquierda=[]
poa_global_lista_izquierda=[]
for x in range(1,len(horas_trayecto)):
    poa_global_filtro_izquierda.append(irrad_izquierda[x-1]['poa_global'])
    df_global_izquierda.append(pd.DataFrame(poa_global_filtro_izquierda[x-1]))
    
for x in range(1,len(horas_trayecto)):   
    poa_global_lista_izquierda.append(df_global_izquierda[x-1].iloc[:, 0]) # Primera columna
    
poa_global_izquierda=[] 
for x in range(1,len(horas_trayecto)): 
    poa_global_izquierda.append(poa_global_lista_izquierda[x-1])
    
#------DATAFRAME--------#
poa_global_izquierda_pd=[]
for x in range(1,len(horas_trayecto)):
    for i in range(0,len(datetimes[x-1])):
        poa_global_izquierda_pd.append(poa_global_izquierda[x-1][i])
df['Izquierda'] = poa_global_izquierda_pd

In [ ]:
pd.set_option("max_columns", None) #Showing only two columns
pd.set_option("max_rows", None)
df

In [ ]:
import matplotlib.pyplot as plt

df.plot(y="Maletero",kind="bar");

In [ ]:
df.plot.area(y='Maletero')
df.plot.area(y='Capó')
df.plot.area(y='Techo')
df.plot.area(y='Derecha')
df.plot.area(y='Izquierda')

In [ ]:
df2=pd.DataFrame(df['Capó'])
df2['Maletero'] = poa_global_maletero_pd
df2['Techo'] = poa_global_techo_pd
df2['Derecha'] = poa_global_derecha_pd
df2['Izquierda'] = poa_global_izquierda_pd
df2.plot();

In [ ]:
df3=pd.DataFrame(df['Azimuth'])
df3.plot()